<a href="https://colab.research.google.com/github/Method-for-Software-System-Development/Cloud_Computing/blob/develop/logic/chatbot_controller.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
OptiBot Chatbot Controller

This module provides the main backend logic for the OptiLine virtual assistant chatbot ("OptiBot").
The chatbot can answer user questions about the OptiLine system, leveraging both generative AI (Gemini) and live data from Firebase when needed.

Functions:
    - ask_optibot(question: str) -> str
        Main entry point. Receives a user question (string) and returns a single English answer as a string.
        Answers can include both real-time data from the system and general knowledge.

Dependencies:
    - google-generativeai (for Gemini API)
    - importnb (for loading other project notebooks as modules)
    - Firebase helper modules (existing: FireBase, user_controller, etc.)
    - Colab secrets (for securely storing Gemini API key)

Note:
    This module is backend-only; it does not include any frontend or UI logic.
    All code comments and docstrings are in English for documentation purposes.
"""

In [ ]:
# ─── Imports ───

import google.generativeai as genai
from google.colab import userdata
import time

# Import project modules using importnb
from importnb import Notebook

# Load Firebase and helper modules as notebooks
with Notebook():
    import FireBase as fb           # Firebase data access functions
    import user_controller as uc    # User management and leaderboard functions
    import mqqt_sim_indoor as indoor    # Indoor sensor simulator
    import mqqt_sim_outdoor as outdoor  # Outdoor sensor simulator

# ─── Constants ───

# The base context provided to the Gemini model for every answer
OPTI_BOT_CONTEXT = """
You are OptiBot – a virtual assistant for the OptiLine system.

OptiLine is a smart cloud-based dashboard for real-time monitoring, fault detection, optimization, and performance tracking in autonomous production lines.

Key features include:
- Real-time sensor data (temperature, speed, accuracy, energy)
- Alerts within 1 second for anomalies
- Gamified optimization challenges (Optimization Race)
- Performance score and team leaderboard
- Intelligent fault response with scoring

Answer all questions as if you are part of the OptiLine platform.
"""

# The Gemini API key is securely retrieved from Colab secrets
GEMINI_API_KEY = userdata.get("GOOGLE_API_KEY")


In [ ]:
def ask_optibot(message: str, chat_history: list) -> tuple:
    """
    Main entry point for OptiBot Q&A.

    Args:
        message (str): The latest user message (question) as a string.
        chat_history (list): A list of previous chat messages.

    Returns:
        tuple:
            - Empty string (to clear the input textbox in Gradio)
            - Updated chat_history (list), including the new user message and the generated bot response.

    Logic flow:
        1. Analyze the incoming message to determine if it requests live sensor data (indoor/outdoor)
           or general system data (leaderboard, faults, etc.).
        2. If live data is needed, fetch relevant data from the proper module (mqqt_sim_indoor/outdoor).
        3. Construct a prompt for the Gemini model that includes:
           - System context
           - (Optionally) real-time data retrieved from the system
           - The user's original question
        4. Call the Gemini API to generate an English response.
        5. Append the user message and the bot response to the chat history.
        6. Return an empty string (to clear the Gradio input box) and the updated chat history.
    """

    # Add the new user message to chat history
    chat_history = chat_history + [[message, None]]
    message_lower = message.lower()
    live_data_note = ""

    # --- Step 1: Pattern matching for live data intent ---

    # Indoor temperature
    if "indoor temperature" in message_lower:
        try:
            indoor_data = next(indoor.get_live_data_stream(mode="simulation"))
            live_data_note = f"Current indoor temperature: {indoor_data['Temperature']} °C."
        except Exception as e:
            live_data_note = f"Unable to retrieve indoor temperature data: {e}"

    # Outdoor temperature
    elif "outdoor temperature" in message_lower:
        try:
            outdoor_data = next(outdoor.get_live_data_stream(mode="simulation"))
            live_data_note = f"Current outdoor temperature: {outdoor_data['Temperature']} °C."
        except Exception as e:
            live_data_note = f"Unable to retrieve outdoor temperature data: {e}"

    # General temperature (prefers outdoor if not specified)
    elif "temperature" in message_lower:
        try:
            outdoor_data = next(outdoor.get_live_data_stream(mode="simulation"))
            live_data_note = f"Current outdoor temperature: {outdoor_data['Temperature']} °C."
        except Exception as e:
            live_data_note = f"Unable to retrieve temperature data: {e}"

    # Leaderboard
    elif "score" in message_lower or "leaderboard" in message_lower:
        top_users, user_rank = uc.get_leaderboard("")
        if top_users:
            table = ", ".join([f"{name} ({score})" for _, name, score in top_users])
            live_data_note = f"Leaderboard: {table}."
        else:
            live_data_note = "Leaderboard data is currently unavailable."

    # Faults
    elif "fault" in message_lower or "error" in message_lower:
        faults = getattr(fc.fb, "get_active_faults", lambda: {})()
        if faults:
            num_faults = len(faults)
            live_data_note = f"There are currently {num_faults} active faults in the system."
        else:
            live_data_note = "No active faults found at the moment."

    # Add more sensor patterns as needed

    # --- Step 2: Build prompt for Gemini ---
    prompt = OPTI_BOT_CONTEXT.strip() + "\n"
    if live_data_note:
        prompt += f"\nRelevant live system data:\n{live_data_note}\n"
    prompt += f"\nUser question: {message}\n"

    # --- Step 3: Call Gemini API ---
    try:
        if not GEMINI_API_KEY:
            bot_response = "Error: Gemini API key is missing. Please contact your administrator."
        else:
            genai.configure(api_key=GEMINI_API_KEY)
            model = genai.GenerativeModel("gemini-1.5-flash")
            response = model.generate_content(prompt)
            bot_response = response.text.strip() if hasattr(response, "text") else str(response)
    except Exception as ex:
        bot_response = f"Sorry, I couldn't process your question due to a system error: {ex}"

    # --- Step 4: Add bot response to chat history and return ---
    chat_history[-1][1] = bot_response
    return "", chat_history
